In [3]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
    log_loss
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.keras
from itertools import product
from random import sample
import webbrowser
import os

# ============================================
# CONFIGURAR MLFLOW
# ============================================

# Opción A: MLflow Local (corre en tu máquina)
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # URI local
experiment_name = "ClasificacionAlejo"

# Opción B: MLflow en Databricks (si usas Databricks)
# mlflow.set_registry_uri("databricks-uc")
# experiment_name = "/ClasificacionAlejo"

# Crear o obtener el experimento
try:
    experiment_id = mlflow.create_experiment(experiment_name)
except:
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id

mlflow.set_experiment(experiment_name)

print(f"Experimento creado/encontrado: {experiment_name}")
print(f"Experiment ID: {experiment_id}")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

# ============================================
# 1. Cargar y preparar los datos
# ============================================
data = pd.read_csv('DatosSingapore.csv')
data.dropna(inplace=True)

# Crear la variable objetivo para clasificación
data['recommended'] = np.where(
    (data['price'] <= 200) & 
    (data['review_scores_rating'] >= 4.5) & 
    (data['bedrooms'] >= 1) &
    (data['amenities_number'] >= 5) &
    (data['host_response_rate'] >= 0.79), 1, 0
)

# Separar características (X) y variable objetivo (y)
X = data.drop(columns=['recommended', 'price'])
y = data['recommended']

# Codificar variables categóricas
X = pd.get_dummies(X, drop_first=True)

# Normalizar las variables numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# ============================================
# DEFINIR ESPACIO DE BÚSQUEDA DE HIPERPARÁMETROS
# ============================================

param_grid = {
    'layer1_units': [64, 128, 256],
    'layer2_units': [32, 64, 128],
    'dropout_rate': [0.2, 0.3, 0.4],
    'learning_rate': [0.001, 0.0001],
    'batch_size': [16, 32, 64],
    'epochs': [50, 100]
}

# Función para crear y entrenar el modelo
def create_and_train_model(params, X_train, y_train, X_test, y_test):
    """
    Crea, entrena y evalúa un modelo con los parámetros especificados
    """
    model = Sequential([
        Dense(params['layer1_units'], input_dim=X_train.shape[1], activation='relu'),
        Dropout(params['dropout_rate']),
        Dense(params['layer2_units'], activation='relu'),
        Dropout(params['dropout_rate']),
        Dense(1, activation='sigmoid')
    ])
    
    # Compilar con learning rate específico
    from tensorflow.keras.optimizers import Adam
    optimizer = Adam(learning_rate=params['learning_rate'])
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=0
    )
    
    # Entrenar
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Predicciones
    y_pred = model.predict(X_test, verbose=0)
    y_pred_classes = (y_pred > 0.5).astype("int32")
    
    # Calcular métricas
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred_classes),
        'precision': precision_score(y_test, y_pred_classes, zero_division=0),
        'recall': recall_score(y_test, y_pred_classes, zero_division=0),
        'f1_score': f1_score(y_test, y_pred_classes, zero_division=0),
        'roc_auc': roc_auc_score(y_test, y_pred),
        'log_loss': log_loss(y_test, y_pred),
        'final_val_loss': history.history['val_loss'][-1],
        'final_val_accuracy': history.history['val_accuracy'][-1],
        'epochs_trained': len(history.history['loss'])
    }
    
    return model, metrics, history

# ============================================
# BÚSQUEDA DE HIPERPARÁMETROS
# ============================================

print("\n" + "="*70)
print("INICIANDO BÚSQUEDA DE HIPERPARÁMETROS")
print("="*70)

all_combinations = list(product(*param_grid.values()))
total_combinations = len(all_combinations)
print(f"Total de combinaciones posibles: {total_combinations}")

# Limitar a N combinaciones aleatorias
n_iterations = min(20, total_combinations)
selected_combinations = sample(all_combinations, n_iterations)
print(f"Combinaciones a probar: {n_iterations}")

best_f1 = 0
best_params = None
best_run_id = None
results = []

for i, combination in enumerate(selected_combinations, 1):
    params = dict(zip(param_grid.keys(), combination))
    
    print(f"\n[{i}/{n_iterations}] Probando: L1={params['layer1_units']}, L2={params['layer2_units']}, " +
          f"Dropout={params['dropout_rate']}, LR={params['learning_rate']}, " +
          f"Batch={params['batch_size']}, Epochs={params['epochs']}")
    
    with mlflow.start_run(
        run_name=f"run_{i:02d}_L{params['layer1_units']}-{params['layer2_units']}_D{params['dropout_rate']}"
    ) as run:
        run_id = run.info.run_id
        
        # Registrar parámetros
        mlflow.log_params(params)
        mlflow.log_param("input_dim", X_train.shape[1])
        mlflow.log_param("iteration", i)
        
        # Tags para organizar
        mlflow.set_tag("model_type", "neural_network")
        mlflow.set_tag("framework", "tensorflow_keras")
        mlflow.set_tag("search_type", "random_search")
        
        # Entrenar modelo
        try:
            model, metrics, history = create_and_train_model(
                params, X_train, y_train, X_test, y_test
            )
            
            # Registrar métricas
            mlflow.log_metrics(metrics)
            
            # Guardar modelo
            mlflow.keras.log_model(model, "model")
            
            # Guardar gráfica de entrenamiento
            fig, axes = plt.subplots(1, 2, figsize=(12, 4))
            axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
            axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
            axes[0].set_xlabel('Época')
            axes[0].set_ylabel('Loss')
            axes[0].set_title('Curva de Pérdida')
            axes[0].legend()
            axes[0].grid(True, alpha=0.3)
            
            axes[1].plot(history.history['accuracy'], label='Train Acc', linewidth=2)
            axes[1].plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
            axes[1].set_xlabel('Época')
            axes[1].set_ylabel('Accuracy')
            axes[1].set_title('Curva de Exactitud')
            axes[1].legend()
            axes[1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig(f'training_curves_{i}.png', dpi=100)
            mlflow.log_artifact(f'training_curves_{i}.png')
            plt.close()
            
            # Actualizar mejor modelo
            if metrics['f1_score'] > best_f1:
                best_f1 = metrics['f1_score']
                best_params = params
                best_run_id = run_id
                mlflow.set_tag("best_so_far", "true")
            
            # Guardar resultados
            result = {
                'run_id': run_id,
                'iteration': i,
                **params,
                **metrics
            }
            results.append(result)
            
            print(f"  ✓ F1={metrics['f1_score']:.4f}, Acc={metrics['accuracy']:.4f}, " +
                  f"ROC-AUC={metrics['roc_auc']:.4f}")
            
        except Exception as e:
            print(f"  ✗ Error en iteración {i}: {str(e)}")
            mlflow.log_param("status", "failed")
            mlflow.log_param("error", str(e))

# ============================================
# RESUMEN DE RESULTADOS
# ============================================

print("\n" + "="*70)
print("RESUMEN DE BÚSQUEDA DE HIPERPARÁMETROS")
print("="*70)

# Crear DataFrame con resultados
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('f1_score', ascending=False)

print("\n📊 Top 5 mejores combinaciones por F1-Score:")
print("-" * 70)
display_cols = ['iteration', 'layer1_units', 'layer2_units', 'dropout_rate', 
                'learning_rate', 'f1_score', 'accuracy', 'roc_auc']
print(results_df[display_cols].head(5).to_string(index=False))

print("\n" + "="*70)
print("🏆 MEJORES HIPERPARÁMETROS ENCONTRADOS:")
print("="*70)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")
print(f"\n  {'F1-Score':20s}: {best_f1:.4f}")
print(f"  {'Run ID':20s}: {best_run_id}")
print("="*70)

# Guardar resultados en CSV
results_csv = 'hyperparameter_search_results.csv'
results_df.to_csv(results_csv, index=False)
print(f"\n✓ Resultados guardados en: {results_csv}")

# ============================================
# ENTRENAR MODELO FINAL CON MEJORES PARÁMETROS
# ============================================

print("\n" + "="*70)
print("ENTRENANDO MODELO FINAL CON MEJORES PARÁMETROS")
print("="*70)

with mlflow.start_run(run_name="BEST_MODEL_FINAL") as final_run:
    final_run_id = final_run.info.run_id
    
    # Tags especiales para el mejor modelo
    mlflow.set_tag("best_model", "true")
    mlflow.set_tag("model_version", "final")
    mlflow.set_tag("selection_criteria", "highest_f1_score")
    
    mlflow.log_params(best_params)
    
    # Entrenar con más épocas para el modelo final
    final_params = best_params.copy()
    final_params['epochs'] = 150
    
    print("Entrenando modelo final...")
    model, metrics, history = create_and_train_model(
        final_params, X_train, y_train, X_test, y_test
    )
    
    # Predicciones finales
    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype("int32")
    
    # Registrar métricas
    mlflow.log_metrics(metrics)
    
    # Matriz de confusión
    plt.figure(figsize=(8, 6))
    conf_matrix = confusion_matrix(y_test, y_pred_classes)
    sns.heatmap(
        conf_matrix,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['No Recomendado', 'Recomendado'],
        yticklabels=['No Recomendado', 'Recomendado'],
        cbar_kws={'label': 'Frecuencia'}
    )
    plt.xlabel('Predicción', fontsize=12)
    plt.ylabel('Valor Real', fontsize=12)
    plt.title('Matriz de Confusión - Modelo Final', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('final_confusion_matrix.png', dpi=150)
    mlflow.log_artifact('final_confusion_matrix.png')
    plt.close()
    
    # Curvas de entrenamiento
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Época', fontsize=11)
    axes[0].set_ylabel('Loss', fontsize=11)
    axes[0].set_title('Curva de Pérdida - Modelo Final', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[1].set_xlabel('Época', fontsize=11)
    axes[1].set_ylabel('Accuracy', fontsize=11)
    axes[1].set_title('Curva de Exactitud - Modelo Final', fontsize=12, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('final_training_curves.png', dpi=150)
    mlflow.log_artifact('final_training_curves.png')
    plt.close()
    
    # Reporte de clasificación
    report = classification_report(
        y_test, y_pred_classes,
        target_names=['No Recomendado', 'Recomendado']
    )
    with open('final_classification_report.txt', 'w', encoding='utf-8') as f:
        f.write("="*60 + "\n")
        f.write("MODELO FINAL - MEJORES HIPERPARÁMETROS\n")
        f.write("="*60 + "\n\n")
        f.write("HIPERPARÁMETROS:\n")
        f.write("-"*60 + "\n")
        for param, value in best_params.items():
            f.write(f"{param:20s}: {value}\n")
        f.write("\n" + "="*60 + "\n")
        f.write("REPORTE DE CLASIFICACIÓN:\n")
        f.write("="*60 + "\n\n")
        f.write(report)
        f.write("\n" + "="*60 + "\n")
        f.write("MÉTRICAS ADICIONALES:\n")
        f.write("-"*60 + "\n")
        f.write(f"ROC-AUC Score:        {metrics['roc_auc']:.4f}\n")
        f.write(f"Log Loss:             {metrics['log_loss']:.4f}\n")
        f.write(f"Épocas entrenadas:    {metrics['epochs_trained']}\n")
    
    mlflow.log_artifact('final_classification_report.txt')
    
    # Guardar modelo final
    mlflow.keras.log_model(model, "final_best_model")
    
    # Crear visualización comparativa
    plt.figure(figsize=(12, 6))
    results_sorted = results_df.sort_values('iteration')
    plt.plot(results_sorted['iteration'], results_sorted['f1_score'], 
             marker='o', linewidth=2, markersize=8, label='F1-Score')
    plt.axhline(y=best_f1, color='r', linestyle='--', linewidth=2, 
                label=f'Mejor F1-Score: {best_f1:.4f}')
    plt.xlabel('Iteración', fontsize=12)
    plt.ylabel('F1-Score', fontsize=12)
    plt.title('Evolución de F1-Score durante Búsqueda de Hiperparámetros', 
              fontsize=13, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('f1_score_evolution.png', dpi=150)
    mlflow.log_artifact('f1_score_evolution.png')
    plt.close()
    
    print("\n" + "="*70)
    print("📈 MÉTRICAS DEL MODELO FINAL")
    print("="*70)
    print(f"  Accuracy:     {metrics['accuracy']:.4f}")
    print(f"  Precision:    {metrics['precision']:.4f}")
    print(f"  Recall:       {metrics['recall']:.4f}")
    print(f"  F1-Score:     {metrics['f1_score']:.4f}")
    print(f"  ROC-AUC:      {metrics['roc_auc']:.4f}")
    print(f"  Log Loss:     {metrics['log_loss']:.4f}")
    print("="*70)

# ============================================
# INFORMACIÓN PARA ACCEDER A MLFLOW UI
# ============================================

print("\n" + "="*70)
print("🎯 CÓMO VISUALIZAR LOS RESULTADOS EN MLFLOW")
print("="*70)

tracking_uri = mlflow.get_tracking_uri()
print(f"\n📍 MLflow Tracking URI: {tracking_uri}")

if tracking_uri.startswith("http://127.0.0.1") or tracking_uri.startswith("http://localhost"):
    print("\n⚠️  IMPORTANTE: Necesitas iniciar el servidor MLflow UI")
    print("\n📝 PASOS PARA VER LA INTERFAZ DE MLFLOW:")
    print("\n1. Abre una nueva terminal/CMD")
    print("2. Ejecuta el siguiente comando:")
    print("\n   mlflow ui")
    print("\n3. Abre tu navegador en: http://127.0.0.1:5000")
    print("\n4. Busca el experimento:", experiment_name)
    
    # Intentar abrir automáticamente (puede no funcionar en todos los sistemas)
    try:
        print("\n🌐 Intentando abrir el navegador automáticamente...")
        webbrowser.open('http://127.0.0.1:5000')
    except:
        print("   (No se pudo abrir automáticamente)")
else:
    print(f"\n🌐 Accede a MLflow UI en: {tracking_uri}")

print("\n" + "="*70)
print("✅ BÚSQUEDA DE HIPERPARÁMETROS COMPLETADA")
print("="*70)
print(f"\n📂 Archivos generados:")
print(f"   • {results_csv}")
print(f"   • final_confusion_matrix.png")
print(f"   • final_training_curves.png")
print(f"   • final_classification_report.txt")
print(f"   • f1_score_evolution.png")
print(f"\n🔑 Run ID del mejor modelo: {final_run_id}")
print("\n" + "="*70)

Experimento creado/encontrado: ClasificacionAlejo
Experiment ID: 166652744160505758
Tracking URI: http://127.0.0.1:5000

INICIANDO BÚSQUEDA DE HIPERPARÁMETROS
Total de combinaciones posibles: 324
Combinaciones a probar: 20

[1/20] Probando: L1=128, L2=128, Dropout=0.2, LR=0.0001, Batch=64, Epochs=100


c:\Users\alejo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/11/28 17:49:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/28 17:49:51 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/28 17:50:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  ✓ F1=0.7981, Acc=0.9047, ROC-AUC=0.9475
🏃 View run run_01_L128-128_D0.2 at: http://127.0.0.1:5000/#/experiments/166652744160505758/runs/0270edc77b504da1a53c7fb6bdcdd54f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/166652744160505758

[2/20] Probando: L1=64, L2=128, Dropout=0.3, LR=0.0001, Batch=16, Epochs=50


c:\Users\alejo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/11/28 17:50:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/28 17:50:22 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/28 17:50:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  ✓ F1=0.8018, Acc=0.9047, ROC-AUC=0.9559
🏃 View run run_02_L64-128_D0.3 at: http://127.0.0.1:5000/#/experiments/166652744160505758/runs/7de89ab0ac6745078055c34806ade1d6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/166652744160505758

[3/20] Probando: L1=128, L2=128, Dropout=0.4, LR=0.001, Batch=16, Epochs=50


...

  ✓ F1=0.7850, Acc=0.8980, ROC-AUC=0.9504
🏃 View run run_20_L128-32_D0.4 at: http://127.0.0.1:5000/#/experiments/166652744160505758/runs/743b870c639149e5bec35f50be8138dc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/166652744160505758

RESUMEN DE BÚSQUEDA DE HIPERPARÁMETROS

📊 Top 5 mejores combinaciones por F1-Score:
----------------------------------------------------------------------
 iteration  layer1_units  layer2_units  dropout_rate  learning_rate  f1_score  accuracy  roc_auc
        11           128           128           0.2         0.0010  0.818605  0.913525 0.949412
        18           256            64           0.2         0.0001  0.818605  0.913525 0.948143
         6           256           128           0.3         0.0010  0.807339  0.906874 0.945794
         3           128           128           0.4         0.0010  0.805825  0.911308 0.955458
         2            64           128           0.3         0.0001  0.801843  0.904656 0.955890

🏆 MEJORES HIPE

c:\Users\alejo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


2025/11/28 18:11:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/28 18:12:00 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/28 18:12:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



📈 MÉTRICAS DEL MODELO FINAL
  Accuracy:     0.9047
  Precision:    0.8037
  Recall:       0.7963
  F1-Score:     0.8000
  ROC-AUC:      0.9453
  Log Loss:     0.2885
🏃 View run BEST_MODEL_FINAL at: http://127.0.0.1:5000/#/experiments/166652744160505758/runs/e9d79ba6f42141fc8f8964438ceaac02
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/166652744160505758

🎯 CÓMO VISUALIZAR LOS RESULTADOS EN MLFLOW

📍 MLflow Tracking URI: http://127.0.0.1:5000

⚠️  IMPORTANTE: Necesitas iniciar el servidor MLflow UI

📝 PASOS PARA VER LA INTERFAZ DE MLFLOW:

1. Abre una nueva terminal/CMD
2. Ejecuta el siguiente comando:

   mlflow ui

3. Abre tu navegador en: http://127.0.0.1:5000

4. Busca el experimento: ClasificacionAlejo

🌐 Intentando abrir el navegador automáticamente...

✅ BÚSQUEDA DE HIPERPARÁMETROS COMPLETADA

📂 Archivos generados:
   • hyperparameter_search_results.csv
   • final_confusion_matrix.png
   • final_training_curves.png
   • final_classification_report.txt
   • f1_score_e